In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import numpy as np
import IPython.display as display
from functools import partial
import matplotlib.pyplot as plt
import glob
import zipfile
from PIL import Image, ImageDraw
from datetime import datetime
from sklearn.model_selection import KFold
import random

In [3]:
is_colab = False
batch_size = 40

In [4]:
tf.random.set_seed(0)
random.seed(0)


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version: ", tf.__version__)

Num GPUs Available:  1
TensorFlow version:  2.15.0


In [6]:
def parse_tfrecord_fn(example_proto):
    # Define the feature description for parsing
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'steering': tf.io.FixedLenFeature([], tf.float32),
    }

    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed_features['image'], channels=3)
    steering = parsed_features['steering']
    image = tf.cast(image, tf.float32)
    # Add random brightness change as data augmentation
    # if random.random() > 0.5:
    #     image = tf.image.random_brightness(image, 0.5)
    # 
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    
    # round steering to 4 decimal places in tensor
    
    return image, steering


In [7]:
def load_dataset(tfrecord_files):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    return raw_dataset.map(parse_tfrecord_fn)

In [8]:
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    if not os.path.exists("datasets"):
        os.makedirs("datasets")
    
    path_to_zip_file = "/content/drive/MyDrive/Colab Notebooks/AD/datasets/dataset_2024-04-0.zip"
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall("./datasets")

    tfrecord_files = list(glob.glob("./datasets/*.tfrecord"))
else:
    tfrecord_files = list(glob.glob("/home/anaya/Develop/autonomous_driving_training/datasets/*.tfrecord"))

In [9]:
def get_model():
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    return tf.keras.Sequential([
        base_model,
        # Flatten the output to feed into the dense layers
        layers.Flatten(),
        layers.Dropout(0.5),
    
        # Second fully connected layer
        layers.Dense(100, activation='relu'),
        layers.BatchNormalization(),
    
        # Third fully connected layer
        layers.Dense(50, activation='relu'),
        layers.BatchNormalization(),
    
        # Fourth fully connected layer
        layers.Dense(10, activation='relu'),
    
        # Output layer
        layers.Dense(1)
    ])

In [10]:
parsed_dataset = load_dataset(tfrecord_files)
shuffled_dataset = parsed_dataset.shuffle(2040)

# Determine split sizes
total_items = sum([1 for _ in shuffled_dataset.as_numpy_iterator()])

num_folds = 4
fold_size = total_items // num_folds

In [11]:
print("Total items: ", total_items)
print("Fold size: ", fold_size)
print("Number of folds: ", num_folds)

Total items:  59514
Fold size:  14878
Number of folds:  4


In [12]:
# NOTE - Restart experiment from here
tf.random.set_seed(0)
random.seed(0)

# Create an array of datasets for each fold
datasets_for_fold = []

for i in range(num_folds):
    start = i * fold_size
    datasets_for_fold.append(
        shuffled_dataset.skip(start).take(fold_size)
    )



In [13]:
log_dir = "./logs/cv/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# Only add the loss to the tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch', profile_batch=2, embeddings_freq=1)

In [14]:
experiments = {
    # "experiment_1": {
    #     "learning_rate": 1e-4,
    #     "weight_decay": 1e-4,
    #     "scheduler": "constant",
    #     "optimizer": "adam"
    # },
    # "experiment_2": {
    #     "learning_rate": 1e-4,
    #     "weight_decay": 1e-5,
    #     "scheduler": "constant",
    #     "optimizer": "adam"
    # },
    # "experiment_3": {
    #     "learning_rate": 1e-4,
    #     "weight_decay": 1e-4,
    #     "scheduler": "step",
    #     "optimizer": "adam"
    # },
    # "experiment_4": {
    #     "learning_rate": 1e-3,
    #     "weight_decay": 1e-3,
    #     "scheduler": "constant",
    #     "optimizer": "adam"
    # },
    # "experiment_5": {
    #     "learning_rate": 1e-3,
    #     "weight_decay": 1e-4,
    #     "scheduler": "constant",
    #     "optimizer": "adam"
    # },
    "experiment_6": {
        "learning_rate": 1e-3,
        "weight_decay": 1e-3,
        "scheduler": "step",
        "scheduler_steps": 5,
        "scheduler_alpha": 0.1,
        "optimizer": "adam"
    },
    "experiment_7": {
        "learning_rate": 1e-3,
        "weight_decay": 1e-4,
        "scheduler": "step",
        "scheduler_steps": 5,
        "scheduler_alpha": 0.1,
        "optimizer": "adam"
    },
    # "experiment_5": {
    #     "learning_rate": 1e-5,
    #     "weight_decay": 1e-6,
    #     "scheduler": "constant",
    #     "optimizer": "adam"
    # },
    # "experiment_5": {
    #     "learning_rate": 1e-4,
    #     "weight_decay": 1e-4,
    #     "scheduler": "step",
    #     "optimizer": "adam"
    # },
    # "experiment_6": {
    #     "learning_rate": 1e-4,
    #     "weight_decay": 1e-5,
    #     "scheduler": "step",
    #     "optimizer": "adam"
    # },
    # "experiment_7": {
    #     "learning_rate": 1e-5,
    #     "weight_decay": 1e-5,
    #     "scheduler": "step",
    #     "optimizer": "sgd"
    # },
    # "experiment_8": {
    #     "learning_rate": 1e-5,
    #     "weight_decay": 1e-5,
    #     "scheduler": "step",
    #     "optimizer": "adam"
    # },
    
}

In [15]:
def meta_scheduler(steps, alpha=0.1):
    def scheduler(epoch, lr):
        if isinstance(steps, int):
            if epoch % steps == 0 and epoch != 0:
                return lr * alpha
            else:
                return lr
        elif isinstance(steps, list):
            if epoch in steps:
                return lr * alpha
            else:
                return lr
        else:
            raise ValueError("Invalid steps parameter")
    return scheduler

In [16]:
from tensorflow.keras import backend as K

def steering_accuracy(threshold=0.05):
    def SA(y_true, y_pred):
        return K.mean(K.cast(K.less_equal(K.abs(y_true - y_pred), threshold), K.floatx()))
    return SA

In [17]:
# Use experimental.sample_from_datasets to approximate K-Fold

def run_cross_validation(experiment_name: str, experiment_params: dict, batch_size: int = 64):
    for i in range(num_folds):
        tf.random.set_seed(0)
        random.seed(0)
    
        train_datasets = [ds for j, ds in enumerate(datasets_for_fold) if j != i]
         # Concatenate the datasets
        train_dataset = train_datasets[0]
        for ds in train_datasets[1:]:
            train_dataset = train_dataset.concatenate(ds)
        
        # Validation dataset
        val_dataset = datasets_for_fold[i]
        
        model = get_model()
        
        # Compile the model (make sure to specify the loss and optimizer)
        adam_optimizer = tf.keras.optimizers.Adam(
            learning_rate=experiment_params["learning_rate"],
            weight_decay=experiment_params["weight_decay"]
        )
        sgd_optimizer = tf.keras.optimizers.SGD(
            learning_rate=experiment_params["learning_rate"],
            weight_decay=experiment_params["weight_decay"],
            momentum=0.9
        )
        
        if experiment_params["optimizer"] == "adam":
            optimizer = adam_optimizer
        elif experiment_params["optimizer"] == "sgd":
            optimizer = sgd_optimizer
        else:
            raise ValueError("Invalid optimizer")
        
        model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[steering_accuracy(threshold=0.05)],
        )
        
        train_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4, min_delta=0.0005)
        val_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, min_delta=0.0005)
        callbacks_list = [tensorboard_callback, train_early_stopping, val_early_stopping]
        
        # Adding scheduler
        if experiment_params["scheduler"] == "step":
            callback_lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
                meta_scheduler(
                    steps=experiment_params["scheduler_steps"],
                    alpha=experiment_params["scheduler_alpha"]
                )
            )
            callbacks_list.append(callback_lr_scheduler)
    
        # Fit the model, add fold number as suffix to the log directory
        print(f"Cross-validation: {experiment_name} on fold {i + 1}")
        print("Params: ", experiment_params)
        tensorboard_callback.log_dir = log_dir + f"/{experiment_name}/fold_{i + 1}"

        model.fit(
            train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE),
            validation_data=val_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE),
            callbacks=callbacks_list,
            epochs=30
        )
       


In [18]:
for experiment_name, experiment_params in experiments.items():
    run_cross_validation(experiment_name, experiment_params, batch_size=batch_size)


Cross-validation: experiment_6 on fold 1
Params:  {'learning_rate': 0.001, 'weight_decay': 0.001, 'scheduler': 'step', 'scheduler_steps': 5, 'scheduler_alpha': 0.1, 'optimizer': 'adam'}
Epoch 1/30


I0000 00:00:1713564205.009246  327588 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1115/1115 [==============================] - 108s 92ms/step - loss: 0.0671 - SA: 0.2777 - val_loss: 0.0468 - val_SA: 0.2621 - lr: 0.0010
Epoch 2/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0165 - SA: 0.4304 - val_loss: 0.0345 - val_SA: 0.3025 - lr: 0.0010
Epoch 3/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0118 - SA: 0.4910 - val_loss: 0.0262 - val_SA: 0.3517 - lr: 0.0010
Epoch 4/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0098 - SA: 0.5253 - val_loss: 0.0258 - val_SA: 0.3945 - lr: 0.0010
Epoch 5/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0089 - SA: 0.5380 - val_loss: 0.0245 - val_SA: 0.3922 - lr: 0.0010
Epoch 6/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0095 - SA: 0.5257 - val_loss: 0.0204 - val_SA: 0.4580 - lr: 1.0000e-04
Epoch 7/30
1115/1115 [==============================] - 103s 92ms/step - loss: 0.0078 - SA: 0.5533 - val_loss: 0.0188 -

KeyboardInterrupt: 

In [ ]:
# def scheduler(epoch, lr):
#     if epoch in (4, 10, 15):
#         return lr * 0.5
#     else:
#         return lr
# 
# callback_lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir ./logs/fit/